In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AnaliseOnibus") \
    .getOrCreate()

print("Spark Session de análise criada.")

df_onibus = spark.read.load("../data/posicoes_onibus_parquet")

print("Tabela Parquet carregada com sucesso!")

df_onibus.createOrReplaceTempView("posicoes")

print("Tabela temporária 'posicoes' criada. Agora podemos usar SQL!")

Spark Session de análise criada.
Tabela Parquet carregada com sucesso!
Tabela temporária 'posicoes' criada. Agora podemos usar SQL!


In [3]:
# Conta todos os registros salvos

total_registros = spark.sql("SELECT COUNT(*) as total FROM posicoes")
total_registros.show()

+-----+
|total|
+-----+
|37945|
+-----+



In [6]:
# 10 linhas de onibus com mais atividade

linhas_mais_ativas = spark.sql("""
    SELECT id_linha, COUNT(*) as ocorrencias
    FROM posicoes
    GROUP BY id_linha
    ORDER BY ocorrencias DESC
    LIMIT 10
""")
linhas_mais_ativas.show()

+--------+-----------+
|id_linha|ocorrencias|
+--------+-----------+
|   34356|        167|
|   33170|        153|
|   32975|        148|
|   33361|        128|
|   33112|        121|
|    2004|        104|
|    1095|        104|
|   33051|        104|
|     900|         99|
|   33626|         97|
+--------+-----------+



In [7]:
# Encontrar a ultima posição registrada de um veículo especifico

prefixo = "66942" 

ultima_posicao = spark.sql(f"""
    SELECT * FROM posicoes 
    WHERE prefixo_veiculo = '{prefixo}' 
    ORDER BY timestamp_veiculo DESC 
    LIMIT 1
""")
ultima_posicao.show()

+-------------------+--------+------------+----------------+-----------------+---------------+---------+-------------------+-------------------+---------+
|  timestamp_captura|id_linha|letreiro_ida|  letreiro_volta|destino_principal|prefixo_veiculo| latitude|          longitude|  timestamp_veiculo|acessivel|
+-------------------+--------+------------+----------------+-----------------+---------------+---------+-------------------+-------------------+---------+
|2025-07-06 20:19:11|   33924|           2|TERM. STO. AMARO|        JD. ORION|          66942|-23.69541|-46.700540000000004|2025-07-06 20:19:11|     true|
+-------------------+--------+------------+----------------+-----------------+---------------+---------+-------------------+-------------------+---------+

